In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
sns.set(style='ticks', palette='RdBu')
#sns.set(style='ticks', palette='Set2')
from functools import reduce
import datetime
import pytz
# import holidays as pyholidays

In [2]:
# paste data from bi

# order = pd.read_excel(r"C:\Users\user\Documents\Alex\Order Efficiency\February 23\Book1.xlsx")


order = pd.read_excel(r"C:\Users\user\Downloads\cda-export (16).xls")
order.head(3)

,doc_entry,Document No.,DateTime,Order Status,Created User,Customer Code,Outlet,Trip Date,Receiving Time,Order Criteria Status,Region,Cutoff Time,Duration
0,1001936,234203251,2023/03/25 12:31:00.000000000,PF & PL Sent to Lens Store,receiving2,10454806,TRM,2023/03/25 00:00:00.000000000,2023/03/25 12:08:00.000000000,Spare Part from HQ,Thika,15,23
1,1000499,232801978,2023/03/25 15:12:00.000000000,Branch Frame Sent to Lens Store,receiving2,10475141,VMA,2023/03/25 00:00:00.000000000,2023/03/25 15:01:00.000000000,Order with New Frame from Branch and Lens from HQ,Thika,15,11
2,1002520,234001248,2023/03/25 15:12:00.000000000,PF Sent to Lens Store,receiving2,10223326,TWO,2023/03/25 00:00:00.000000000,2023/03/25 15:01:00.000000000,Order with PF from Branch and Lens from HQ,Thika,15,11


In [3]:
order['Receiving Time'] = pd.to_datetime(order['Receiving Time'],format ='%Y/%m/%d %H:%M:%S')
order.head(2)

,doc_entry,Document No.,DateTime,Order Status,Created User,Customer Code,Outlet,Trip Date,Receiving Time,Order Criteria Status,Region,Cutoff Time,Duration
0,1001936,234203251,2023/03/25 12:31:00.000000000,PF & PL Sent to Lens Store,receiving2,10454806,TRM,2023/03/25 00:00:00.000000000,2023-03-25 12:08:00,Spare Part from HQ,Thika,15,23
1,1000499,232801978,2023/03/25 15:12:00.000000000,Branch Frame Sent to Lens Store,receiving2,10475141,VMA,2023/03/25 00:00:00.000000000,2023-03-25 15:01:00,Order with New Frame from Branch and Lens from HQ,Thika,15,11


In [4]:
order['Hour'] = order['Receiving Time'].dt.strftime('%H')
order.head(1)

,doc_entry,Document No.,DateTime,Order Status,Created User,Customer Code,Outlet,Trip Date,Receiving Time,Order Criteria Status,Region,Cutoff Time,Duration,Hour
0,1001936,234203251,2023/03/25 12:31:00.000000000,PF & PL Sent to Lens Store,receiving2,10454806,TRM,2023/03/25 00:00:00.000000000,2023-03-25 12:08:00,Spare Part from HQ,Thika,15,23,12


In [5]:
order['Delay'] = np.where(order['Duration']>order['Cutoff Time'],1,0) 
order.head(1)

,doc_entry,Document No.,DateTime,Order Status,Created User,Customer Code,Outlet,Trip Date,Receiving Time,Order Criteria Status,Region,Cutoff Time,Duration,Hour,Delay
0,1001936,234203251,2023/03/25 12:31:00.000000000,PF & PL Sent to Lens Store,receiving2,10454806,TRM,2023/03/25 00:00:00.000000000,2023-03-25 12:08:00,Spare Part from HQ,Thika,15,23,12,1


In [6]:
order.to_clipboard()

## Town Branches

In [7]:
town = order[order['Region'] != 'Upcountry']

In [8]:
town['Region'].unique()

array(['Thika', 'Westlands', 'Mombasa Road', 'Eastlands', 'Karen',
       'Rongai', 'CBD'], dtype=object)

In [9]:

town1 = pd.pivot_table(town, index = ['Hour'], values=['doc_entry'], aggfunc='count', fill_value=0, margins=True, margins_name='Total')
town2 = pd.pivot_table(town, index=['Hour'], values=['Duration'], aggfunc = 'mean', fill_value=0, margins=True, margins_name='Total')
town3 = pd.pivot_table(town, index=['Hour'], values=['Delay'], aggfunc = 'sum', fill_value=0, margins=True, margins_name='Total')


In [10]:
town4 = pd.merge(town1,town2, on=['Hour'], how = 'left')
town4

,doc_entry,Duration
Hour,,
10,6,7.666667
11,1,11.000000
12,43,41.906977
13,9,8.333333
14,4,20.250000
15,26,11.192308
16,1,9.000000
Total,90,25.722222


In [11]:
town_pivot = pd.merge(town4,town3, on=['Hour'], how = 'left')
town_pivot

,doc_entry,Duration,Delay
Hour,,,
10,6,7.666667,0
11,1,11.000000,0
12,43,41.906977,35
13,9,8.333333,1
14,4,20.250000,1
15,26,11.192308,5
16,1,9.000000,0
Total,90,25.722222,42


In [12]:
town_pivot.columns

Index(['doc_entry', 'Duration', 'Delay'], dtype='object')

In [13]:
town_pivot['% of Efficiency'] = (town_pivot['doc_entry'] - town_pivot['Delay'])/town_pivot['doc_entry']
town_pivot['Order count'] = town_pivot['doc_entry']
town_pivot['Time Min'] =town_pivot['Duration'].round(2)
town_pivot['% of Efficiency'] =town_pivot['% of Efficiency'].map('{:.2%}'.format)
town_pivot = town_pivot[['Order count','Time Min','Delay','% of Efficiency']]
town_pivot = np.transpose(town_pivot)
town_pivot

Hour,10,11,12,13,14,15,16,Total
Order count,6,1,43,9,4,26,1,90
Time Min,7.67,11.0,41.91,8.33,20.25,11.19,9.0,25.72
Delay,0,0,35,1,1,5,0,42
% of Efficiency,100.00%,100.00%,18.60%,88.89%,75.00%,80.77%,100.00%,53.33%


## Upcountry Branches

In [14]:
upcountry = order[order['Region'] == 'Upcountry']
upcountry['Region'].unique()

array(['Upcountry'], dtype=object)

In [15]:
upcountry1 = pd.pivot_table(upcountry, index = ['Hour'], values=['doc_entry'], aggfunc='count', fill_value=0, margins=True, margins_name='Total')
upcountry2 = pd.pivot_table(upcountry, index=['Hour'], values=['Duration'], aggfunc = 'mean', fill_value=0, margins=True, margins_name='Total')
upcountry3 = pd.pivot_table(upcountry, index=['Hour'], values=['Delay'], aggfunc = 'sum', fill_value=0, margins=True, margins_name='Total')


In [16]:
upcountry4 = pd.merge(upcountry1,upcountry2, on=['Hour'], how = 'left')
upcountry4

,doc_entry,Duration
Hour,,
09,53,83.377358
Total,53,83.377358


In [17]:
upcountry_pivot = pd.merge(upcountry4,upcountry3, on=['Hour'], how = 'left')
upcountry_pivot

,doc_entry,Duration,Delay
Hour,,,
09,53,83.377358,49
Total,53,83.377358,49


In [18]:
upcountry_pivot['% of Efficiency'] = (upcountry_pivot['doc_entry'] - upcountry_pivot['Delay'])/upcountry_pivot['doc_entry']
upcountry_pivot['Order count'] = upcountry_pivot['doc_entry']
upcountry_pivot['Time Min'] =upcountry_pivot['Duration'].round(2)
upcountry_pivot['% of Efficiency'] =upcountry_pivot['% of Efficiency'].map('{:.2%}'.format)
upcountry_pivot = upcountry_pivot[['Order count','Time Min','Delay','% of Efficiency']]
upcountry_pivot = np.transpose(upcountry_pivot)
upcountry_pivot

Hour,09,Total
Order count,53,53
Time Min,83.38,83.38
Delay,49,49
% of Efficiency,7.55%,7.55%


In [19]:
import xlsxwriter

In [20]:
with pd.ExcelWriter(r"C:\Users\user\Documents\Alex\Order Efficiency\November\results\order efficiency RECEIVING.xlsx", engine='xlsxwriter') as writer:
    town_pivot.to_excel(writer,sheet_name='Efficiency')
    upcountry_pivot.to_excel(writer,sheet_name='Efficiency', startrow=8)